<a href="https://colab.research.google.com/github/MMRES-PyBootcamp/MMRES-python-bootcamp2024/blob/master/08_scipy_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 8 - Statistics in Python (Second part - 50')


> A very (very) basic introduction on statistics in Python. In this first introductory lesson we will just present some *Measures of Central Tendency* (median, mean and weighted mean) and *Measures of Variability* (variance and standard deviation). We will also talk about *Percentiles* and *Missing values*.
> 
TODO

sources:

https://medium.com/insights-school/learn-basic-statistics-with-python-cc0f45275929

https://scipy-lectures.org/intro/scipy.html#scipy


In [ ]:
import math
import statistics
from scipy import stats
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# TODO
import plotnine as p9

## Outline
 * [Loading a dataset](#Loading-a-dataset) 
 * [Pearson correlation](#Pearson-correlation)
 * [Spearman correlation](#Spearman-correlation)
 * [Linear regression](#Linear-regression)
 * [Student's t-test](#Student's-t-test)
   * [One-sample t-test](#One-sample-t-test)
   * [Two-sample t-test (Independent samples t-test)](#Two-sample-t-test-(Independent-samples-t-test))
   * [Paired-samples t-test (Dependent samples t-test)](#Paired-samples-t-test-(Dependent-samples-t-test))

Linear models, multiple factors, and analysis of variance

Categorical variables

Link to t-tests between different FSIQ and PIQ

Multiple Regression

Analysis of variance (ANOVA)

This document is devised as a tool to enable your **self-learning process**. If you get stuck at some step or need any kind of help, please don't hesitate to raise your hand and ask for the teacher's guidance. Along it, you will find some **special cells**:

<div class="alert alert-block alert-success"><b>Practice:</b> Practice cells announce exercises that you should try during the current boot camp session. Usually, solutions are provided using hidden cells (look for the dot dot dot symbol "..." and click to unravel them and check that your try is correct).
</div>

<div class="alert alert-block alert-warning"><b>Extension:</b> Extension cells correspond to exercises (or links to contents) that are a bit more advanced. We recommend to try them after the current boot camp session.
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> Tip cells just give some advice or complementary information.
</div>

<div class="alert alert-block alert-danger"><b>Caveat:</b> Caveat cells warn you about the most common pitfalls one founds when starts his/her path learning Python.

</div>

## Loading a dataset

Let's stats by loading a dataset from this cool [online resource](https://lectures.scientific-python.org/packages/statistics/index.html).

In [ ]:
# Loading the "Brain Size" dataset directly from an URL
df = pd.read_csv(
    filepath_or_buffer='https://scipy-lectures.org/_downloads/brain_size.csv',
    sep=';',
    na_values=".",
    index_col=0
)

# Show df's head
df.head()

The columns *FSIQ*, *VIQ* and *PIQ* stand for different "intelligence quotients" (in arbitrary units I guess) and the *MRI_Count* is another "intelligence" measure based on magnetic resonance imaging (also in some kind of arbitrary unit I guess). The columns *Gender*, *Weight* and *Height* express the gender, the weight (in pounds) and the height (in inches) of the individuals, respectively. Before starting, let's express *Weight* and *Height* column values to metric units to avoid an eventual [Mars probe crash](https://www.simscale.com/blog/nasa-mars-climate-orbiter-metric/).

In [ ]:
# Convert from imperial units to metric units
df['Weight_kg'] = df['Weight'] * 0.45359237
df['Height_m'] = df['Height'] * 0.0254

# Drop imperial columns
df.drop(columns=['Weight', 'Height'], inplace=True)

# Show df's head
df.head()

## Pearson correlation
We can compute the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) using the [`pearsonr()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html) function from **SciPy**'s stats module. This function takes two arrays and returns a tuple containing **Pearson correlation coefficient** and the **significance of the correlation** as p-value.

In [ ]:
# Get Pearson between 'FSIQ' and 'VIQ' using SciPy
stats.pearsonr(x=df['FSIQ'], y=df['VIQ'])

We can also get the Pearson correlation coefficient using the [`.corr()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) **Pandas** method:

In [ ]:
# Get Pearson between 'FSIQ' and 'VIQ' using Pandas
df['FSIQ'].corr(df['VIQ'], method="pearson")

Look what happens when we apply this `.corr()` method to a Dataframe:

In [ ]:
# Get Pearson for all pair-wise pairs 'FSIQ' and 'VIQ' using Pandas
df[['FSIQ', 'VIQ']].corr(method='pearson')

By using the **Pandas** method `corr()` on a dataframe, we obtain all possible pair-wise correlations nicely displayed in a "square" dataframe (or matrix). Let's try this strategy again on the whole `df` dataframe (note the trick of the `numeric_only=` argument to avoid the error that pops up when trying to compute correlations involving the `'Gender'` column):

In [ ]:
# Get all pair-wise Pearson from our df
df_PCC = df.corr(method='pearson', numeric_only=True)

# Show the Pearson correlation matrix
df_PCC

Now let's leverage Seaborn [`.heatmap()`](https://seaborn.pydata.org/generated/seaborn.heatmap.html) to visualize this!

In [ ]:
# Plot the correlation matrix (HEATMAP!)
sns.heatmap(data=df_PCC, cmap="Blues", annot=True)

It seems that *FSIQ*, *VIQ* and *PIQ* are very correlated, and similarly, *Weight [kg]*, *Height [m]* and (to a lesser extend) *MRI_Count*.

## Spearman correlation

Pearson correlation assumes that the data we are comparing is <u>**normally distributed**</u>. When this assumption is not true, the Pearson correlation value is not reflecting the true association. Spearman correlation does not assume that data follows a specific distribution, so it is a non-parametric correlation measure. Spearman correlation is also known as Spearman's rank correlation as it computes correlation coefficient on rank values of the data. Using **SciPy**, we can compute Spearman correlation using the function `spearmanr()`:

In [ ]:
#The first element of tuple is the Spearman's rank correlation and the second is p-value.
stats.spearmanr(a=df['FSIQ'], b=df['VIQ'])

<div class="alert alert-block alert-info">

<b> Tip:</b> I don't know why `stats.pearsonr()` uses `x=` and `y=` as argument names and `stats.spearmanr()` uses `a=` and `b=`. In any case, you can use these two functions without specifying argument names: `stats.pearsonr(df['FSIQ'], df['VIQ'])` or `stats.spearmanr(df['FSIQ'], df['VIQ'])`.

</div>

Again, we can also get the Spearman correlation coefficient using the [`.corr()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) **Pandas** method:

In [ ]:
# Get Spearman between 'FSIQ' and 'VIQ' using Pandas
df['FSIQ'].corr(df['VIQ'], method="spearman")

Now, we will use the Seaborn function [`clustermap()`](https://seaborn.pydata.org/generated/seaborn.clustermap.html) instead of `heatmap()`:

In [ ]:
# Get all pair-wise Spearman from our df
df_ECC = df.corr(method='spearman', numeric_only=True)

# Show the Spearman correlation matrix
df_ECC

# Plot the Spearman correlation matrix (CLUSTERMAP!)
sns.clustermap(data=df_ECC, cmap="Reds", annot=True, z_score=None, figsize=(6,6))

Have a look at the [dendrogram](https://en.wikipedia.org/wiki/Dendrogram) and note how it hierarchizes closer those variables more correlated.

## Linear regression

With the **SciPy** function [`linregress()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html) it's very easy to perform a linear regression and get all required coefficients.

In [ ]:
# Compute a linear regression between 'FSIQ' and 'VIQ' using SciPy
stats.linregress(x=df['FSIQ'], y=df['VIQ'])

Note how `stats.linregress()` returns an object storing five values: `slope`, `intercept`, `rvalue`, `pvalue`, `stderr` and `intercept_stde`. You can assign the output of `stats.linregress()` in a variable and easily access its elements as follows:

In [ ]:
# Compute a linear regression between 'FSIQ' and 'VIQ' using SciPy
FitResults = stats.linregress(x=df['FSIQ'], y=df['VIQ'])

# Return the 'rvalue' (Pearson correlation) generated by the linear regression
FitResults.rvalue

By using some f-string trickery we can easily prepare a nice annotation with the regression line equation:

In [ ]:
# Gather the regression line equation (using just 3 decimal places with the :.3f trick)
annotation = f'y(x) = {FitResults.slope:.3f} x + {FitResults.intercept:.3f}'

# Show the regression line equation
print(annotation)

Now, let's use **Seaborn** to visualize the regression line along with the data:

In [ ]:
# Create the scatter plot with Seaborn
p = sns.scatterplot(data=df, x="FSIQ", y="VIQ", hue="Gender")

# Add the linear regression line (by hand) with Matplotlib
p.axline(xy1=(0, FitResults.intercept), slope=FitResults.slope, color='lightgray')

# Adjust scatter plot limits (by hand) with Matplotlib
p.set_xlim(df["FSIQ"].min()-2, df["FSIQ"].max()+2)
p.set_ylim(df["VIQ"].min()-2, df["VIQ"].max()+2) 

# Add regression line equation (by hand) with Matplotlib
plt.text(95, 145, annotation)

With **Plotnine** we can also get a very nice plot (maybe with less tweaking):

In [ ]:
gg = (
    
    # Create the scatter plot with Plotnine
    p9.ggplot(data=df, mapping=p9.aes(x="FSIQ", y="VIQ", color="Gender"))
     + p9.geom_point()

    # Add the linear regression line (by hand)
     + p9.geom_abline(intercept=FitResults.intercept, slope=FitResults.slope, color='lightgray')

    # Add regression line equation (by hand)
     + p9.annotate(geom="text", x=95, y=145, label=annotation)
)

# Show plot
gg

## Student's t-test

A t-test is a statistical tool used to determine if there is a significant difference between the means of two groups, or between the mean of one group and a known value. There are three kinds of t-tests: **One-Sample t-test**, **Two-sample t-test** and **Paired samples t-test**.

### One-sample t-test

The one-sample t-test is used to **compare the mean of a single group to a known or hypothesized value**. This test is appropriate when you want to determine if a sample comes from a population with a specific mean. **SciPy** has implemented a function to perform one-sample t-test: [`scipy.stats.ttest_1samp()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html). Let's see an example:

In [ ]:
# Represent the distribution traced by VIQ and mark its mean
sns.kdeplot(data=df, x="VIQ", color='blue', fill='blue', alpha=0.1)
plt.axvline(x=df['VIQ'].mean(), color='blue')

# Mark a couple of arbitrary values
plt.axvline(x=100, color='red')
plt.axvline(x=105, color='green')

To test if the difference between the mean of *VIQ* and $105$ is significant, we should do:

In [ ]:
# Launching a 1STT to check if VIQ mean differs than 105
stats.ttest_1samp(a=df['VIQ'], popmean=105)

This function returns an object with the **T statistic**, the **p-value** and the **degrees of freedom**. If, for example, we assume a significance level of $0.05$, we cannot say that the mean of *VIQ* is different than $105$. In contrast, with this same significance level, we can say that the mean of *VIQ* is different than $100$:

In [ ]:
# Launching a 1STT to check if VIQ mean differs than 100
stats.ttest_1samp(a=df['VIQ'], popmean=100)

### Two-sample t-test (Independent samples t-test)

The two-sample t-test is used to **compare the means of two unrelated groups**. The term "unrelated" means that the two groups are not related to each other (different patients with and without a given clinical condition for example). **SciPy** has implemented a function to perform two-sample t-test: [`scipy.stats.ttest_ind()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html). For example, let's compare the mean *VIQ* between females and males:

In [ ]:
# Group by 'Gender' and get the mean VIQ
df_g = df.groupby(by='Gender').agg(Mean_VIQ=('VIQ', 'mean'))

# Show the data frame
df_g

In [ ]:
# Represent the distribution traced by VIQ differentiating males and females
sns.kdeplot(data=df, x="VIQ", hue='Gender', fill='Gender')

# Mark the mean VIQ for males (orange) and females (blue)
plt.axvline(x=df_g['Mean_VIQ']['Male'], color=sns.color_palette()[0])
plt.axvline(x=df_g['Mean_VIQ']['Female'], color=sns.color_palette()[1])
# Note how we used the same default colors used by Seaborn with sns.color_palette()[ ] ;)

It seems that the mean *VIQ* is slightly different between genders, but is this difference *statistically significant*? Let's check it out!

In [ ]:
# Prepare filters for females and males
filter_F = df['Gender'] == 'Female'
filter_M = df['Gender'] == 'Male'

# Launching a 2STT to check if VIQ mean differs between females and males
stats.ttest_ind(df[filter_F]['VIQ'], df[filter_M]['VIQ'])

Assuming again the typical significance level of $0.05$, we cannot report a significant *VIQ* different between genders.

### Paired-samples t-test (Dependent samples t-test)

Finally, the paired samples t-test is used to **compare the means of two related groups**. Here, the term "related" denotes that the groups are not independent (same patients before and after a treatment for example). **SciPy** implements a function to perform paired-samples t-test: [`scipy.stats.ttest_rel()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html). In the previous example we compared *VIQ* between females and males (independent groups). Now, let's compare the three IQ estimators provided in our data set (*PIQ*, *VIQ*, and *FSIQ*).

<div class="alert alert-block alert-info">

<b>Tip:</b> We want to visualize the three IQ estimators in a same plot to better glimpse whats going on. In order to easily do that leveraging Seaborn, we first need to tweak a bit the original data frame `df` by melting it. By the way, we will also get the mean for each IQ estimator as we just did to get the mean <i>VIQ</i> for females and males.

</div>

In [ ]:
# Melting our data frame to better visualize it with Seaborn
df_m = pd.melt(frame=df, id_vars=None, value_vars=['PIQ', 'VIQ', 'FSIQ'], var_name='IQ type', value_name='IQ value') 

# Show the data frame
df_m

In [ ]:
# Group by 'IQ type' and get the mean IQ
df_g = df_m.groupby(by='IQ type').agg(Mean_IQ=('IQ value', 'mean'))

# Show the data frame
df_g

In [ ]:
# Represent the distribution traced by each IQ type
sns.kdeplot(data=df_m, x="IQ value", hue='IQ type', fill='IQ type')

# Mark the mean IQ for each IQ type: PIQ (blue), VIQ (orange), FSIQ (green)
plt.axvline(x=df_g['Mean_IQ']['PIQ'], color=sns.color_palette()[0])
plt.axvline(x=df_g['Mean_IQ']['VIQ'], color=sns.color_palette()[1])
plt.axvline(x=df_g['Mean_IQ']['FSIQ'], color=sns.color_palette()[2])
# Note how we used the same default colors used by Seaborn with sns.color_palette()[ ] ;)

At first sight, all three IQ estimators exhibit quite alike bimodal distributions with very similar means for `IQ value`. Now, let's check it out!

In [ ]:
# Launching a PSTT to check if PIQ mean differs from VIQ mean
print(stats.ttest_rel(df['PIQ'], df['VIQ']))

# Launching a PSTT to check if PIQ mean differs from FSIQ mean
print(stats.ttest_rel(df['PIQ'], df['FSIQ']))

# Launching a PSTT to check if VIQ mean differs from FSIQ mean
print(stats.ttest_rel(df['VIQ'], df['FSIQ']))

Assuming again a significance level of $0.05$, we cannot report a significant difference between the three IQ estimators available.

<div class="alert alert-block alert-info">
    
<b>Tip:</b> Imagine that (instead of three IQ estimators) you have, let's say, one hundred. Getting all possible pairs and writing the corresponding statistical tests by hand could be dangerous. In these situations you should <i>loop</i> along all possible pair combinations:

</div>

In [ ]:
# Load the itertools package
import itertools

# Define a list with all our IQ estimators
list_IQs = ['PIQ', 'VIQ', 'FSIQ']

# Get a list with all possible list_IQs combinations pairs (repeat=2)
list_IQ_pairs = itertools.combinations(iterable=list_IQs, r=2) 

# Loop along all possible pairs...
for pair in list_IQ_pairs:
    
    #... print the running pair and its corresponding paired samples t-test outcome
    print(pair, stats.ttest_rel(df[pair[0]], df[pair[1]]), '\n')

<div class="alert alert-block alert-warning">
    
<b>Extension:</b> You can also achieve this with a **list comprehension** instead of a **for loop**:

</div>

In [ ]:
# Define a list with all our IQ estimators
list_IQs = ['PIQ', 'VIQ', 'FSIQ']

# Get a list with all possible list_IQs combinations pairs (repeat=2)
list_IQ_pairs = itertools.combinations(iterable=list_IQs, r=2) 

# Give a tuple ( , ) with the running pair and its corresponding paired samples t-test outcome for all possible pairs
[( pair, stats.ttest_rel(df[pair[0]], df[pair[1]]) ) for pair in list_IQ_pairs]

<div class="alert alert-block alert-success">
    
<b>Practice 1:</b> Test if the weight for females and males is different using a t-test.

1) Which t-test will you use (one-, two- or paired-sample t-test)? Why?
2) In the 1<sup>st</sup> code cell below, use the chosen t-test to statistically evaluate the weight difference between females and males. What happened in this first try?
3) In the 2<sup>nd</sup> code cell below, try again with the `stats.ttest_***()` argument `nan_policy='omit'`. What happened in this second try? What was the outcome of your t-test?
4) In the 3<sup>rd</sup> code cell below, prepare a nice plot showing the weight distribution for females and males, marking their corresponding mean values with vertical lines.

<b>Note:</b> Uncomment and fill only those code lines with underscores `___`.

</div>

In [ ]:
# (1st code cell)

# Prepare filters for females and males
#filter_F = ___
#filter_M = ___

# Launching a 2STT to check if the weight mean differs between females and males
#stats.___(___, ___)

In [ ]:
# (1st code cell --> SOLUTION)

# Prepare filters for females and males
filter_F = df['Gender'] == 'Female'
filter_M = df['Gender'] == 'Male'

# Launching a 2STT to check if the weight mean differs between females and males
stats.ttest_ind(df[filter_F]['Weight [kg]'], df[filter_M]['Weight [kg]'])

In [ ]:
# (2nd code cell)

# Launching a 2STT to check if the weight mean differs between females and males
#stats.___(___, ___, ___)

In [ ]:
# (2nd code cell --> SOLUTION)

# Launching a 2STT to check if the weight mean differs between females and males
stats.ttest_ind(df[filter_F]['Weight [kg]'], df[filter_M]['Weight [kg]'], nan_policy='omit')

In [ ]:
# (3rd code cell)

# Group by 'Gender' and get the mean weight
#df_g = df.groupby(by=___).agg(Mean_Weight_kg=(___, ___))

# Show the data frame
#print(df_g)

# Represent the distribution traced by weight differentiating males and females
#sns.kdeplot(data=___, x=___, hue=___, fill=___)

# Mark the mean weight for males (orange) and females (blue)
#plt.axvline(x=___['Mean_Weight_kg'][___], color=sns.color_palette()[___])
#plt.axvline(x=___['Mean_Weight_kg'][___], color=sns.color_palette()[___])
# Note how we used the same default colors used by Seaborn with sns.color_palette()[ ] ;)

In [ ]:
# (3rd code cell --> SOLUTION)

# Group by 'Gender' and get the mean weight
df_g = df.groupby(by='Gender').agg(Mean_Weight_kg=('Weight [kg]', 'mean'))

# Show the data frame
print(df_g)

# Represent the distribution traced by weight differentiating males and females
sns.kdeplot(data=df, x='Weight [kg]', hue='Gender', fill='Gender')

# Mark the mean weight for males (orange) and females (blue)
plt.axvline(x=df_g['Mean_Weight_kg']['Female'], color=sns.color_palette()[0])
plt.axvline(x=df_g['Mean_Weight_kg']['Male'], color=sns.color_palette()[1])
# Note how we used the same default colors used by Seaborn with sns.color_palette()[ ] ;)

<div class="alert alert-block alert-success"><b>Practice 1 ends here.</b>

</div>

# Linear models, multiple factors, and analysis of variance
Given two set of observations, x and y, we want to test the hypothesis that y is a linear function of x. In other terms:

> y = ax + b + e

where e is observation noise. We will use the statsmodels module to:

Fit a linear model. We will use the simplest strategy, ordinary least squares (OLS).
Test that coef is non zero.

In [ ]:
x = np.linspace(-5, 5, 20)
x2= np.linspace(0,10, 20)
x3=  np.random.normal(size=20)

np.random.seed(1)
# normal distributed noise
y = -5 + 3*x -10*x2 + 4 * np.random.normal(size=x.shape)
# Create a data frame containing all the relevant variables
data = pd.DataFrame({'x': x,'x2': x2, 'x3':x3, 'y': y})

In [ ]:
-5 + 4*np.random.normal(size=x.shape).mean()

In [ ]:
data

In [ ]:
model = ols("Weight_kg ~ Height_m + 1", df).fit()
print(model.summary())

In [ ]:
sns.scatterplot(data=df, x='Height_m', y='Weight_kg')

In [ ]:
from statsmodels.formula.api import ols
model = ols("y ~ x + x2 + x3", data).fit()
print(model.summary())

## Categorical variables:
 comparing groups or multiple categories


In [ ]:
data = pd.read_csv('https://scipy-lectures.org/_downloads/brain_size.csv', sep=';', na_values=".")
model = ols("VIQ ~ Gender + 1", data).fit()
print(model.summary())


## Link to t-tests between different FSIQ and PIQ

To compare different types of IQ, we need to create a “long-form” table, listing IQs, where the type of IQ is indicated by a categorical variable:

In [ ]:
data_fisq = pd.DataFrame({'iq': data['FSIQ'], 'type': 'fsiq'})
data_piq = pd.DataFrame({'iq': data['PIQ'], 'type': 'piq'})
data_long = pd.concat((data_fisq, data_piq))
print(data_long)

#sns.boxplot(data=data_long,x='type',y='iq')
sns.violinplot(data=data_long,x='type',y='iq')



In [ ]:
model = ols("iq ~ type", data_long).fit()
print(model.summary())


## Multiple Regression:
including multiple factors

In [ ]:
data = pd.read_csv('https://scipy-lectures.org/_downloads/iris.csv')
model = ols('sepal_width ~ name + petal_length', data).fit()
print(model.summary())

# Analysis of variance (ANOVA)
In the above iris example, we wish to test if the petal length is different between versicolor and virginica, after removing the effect of sepal width. This can be formulated as testing the difference between the coefficient associated to versicolor and virginica in the linear model estimated above (it is an Analysis of Variance, ANOVA). For this, we write a vector of ‘contrast’ on the parameters estimated: we want to test "name[T.versicolor] - name[T.virginica]", with an F-test.

In [ ]:
print(model.f_test([0, 1, -1, 0]))